In [34]:
import os
import subprocess

os.makedirs("downloads", exist_ok=True)
issues = ["a", "b", "c", "d", "e", "f", "g"]
for issue in issues:
    if not os.path.exists(f"downloads/measure-{issue}.html"):
        subprocess.Popen(["wget", f"https://voterguide.sfelections.org/local-ballot-measures/measure-{issue}", "-O", f"downloads/measure-{issue}.html"])
    

In [35]:
import json 
import time

def wait_on_run(run, thread, client):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [36]:
from openai import OpenAI
issue = "a"

client = OpenAI()
debate_instructions="A debate is an organized argument or contest of ideas in which the \
    participants discuss a topic from two opposing sides.  Each side will show in an organized \
    and clever way why they believe to have the right answers.  They will use examples and \
    evidence to support their ideas while working towards a conclusion. \
    The aim of a debate is to convince the opposition that you are right. \
    When the two sides agree on the subject or when one side's arguments are more convincing \
    than the other side that is when the debate comes to a close. \
    Be concise and informative, do not repeat information already stated, and admit when you are wrong and when the other debater is right. \
    Respond to the Moderator's questions and to the other debater's arguments. "

# Upload the file that you want
file = client.files.create(file=open(f"downloads/measure-{issue}.html", "rb"), purpose="assistants")
# Make your debaters
debater_1 = client.beta.assistants.create(
    name="Debater 1",
    instructions=debate_instructions + f". You support Measure {issue}. You are debating someone who oppose it. Preface each of your answers with Debater 1:",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id]
)
debater_2 = client.beta.assistants.create(
    name="Debater 2",
    instructions=debate_instructions + f". You oppose Measure {issue}. You are debating someone who supports it. Preface each of your answers with Debater 2:",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id]
)


In [37]:
# Opening prompt
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user", 
    content = f"Moderator: Please concisely summarize Measure {issue} and your stance on it"
)

In [38]:
# Debate
num_rounds = 3

# Let em at it!
for i in range(num_rounds):
    for debater in [debater_1, debater_2]:
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=debater.id,
        )
        run = wait_on_run(run, thread, client)
        messages = client.beta.threads.messages.list(thread_id=thread.id)
    message = client.beta.threads.messages.create(
        thread_id=thread.id, 
        role="user", 
        content = f"Moderator: Please respond to the other debator's arguments and introduce novel arguments to make your case"
    )


# Closing remarks
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user", 
    content = f"Moderator: Summarize pro's and con's raied on {issue} and whether you have changed your mind"
)

for msg in messages.data:
    print(msg.content[0].text.value)


Debater 2: Measure A, known as the San Francisco Affordable Housing Bonds, is a proposal that aims to use $300,000,000 in general obligation bonds to fund the development of housing that will be affordable to households ranging from extremely low-income to moderate-income levels, including specific provisions for senior housing and workforce housing. While the intentions behind Measure A may be noble, the practical implications of issuing such a large amount of bonds raise significant concerns.

First and foremost, the financing of these bonds through increased taxes on property could lead to a rise in living costs for all San Franciscans. An estimated average tax rate increase of $0.0057/$100 of assessed property value, while seemingly low, contributes to the overall tax burden and can compound over time, affecting both homeowners and renters. Landlords may pass on half of these tax increases to tenants, exacerbating the affordability crisis it seeks to address.

Moreover, there are r